## General

In [ ]:
from brainlit.map_neurons.map_neurons import (
    DiffeomorphismAction,
    transform_geometricgraph,
    compute_derivs,
    CloudReg_Transform,
)
from brainlit.algorithms.trace_analysis.fit_spline import (
    GeometricGraph,
    compute_parameterization,
)
from brainlit.utils.Neuron_trace import NeuronTrace

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from scipy.ndimage import gaussian_filter
from scipy.interpolate import splev, splprep, RegularGridInterpolator
from scipy.spatial.distance import cosine
from scipy.stats import wilcoxon, pearsonr, linregress, norm
from tqdm import tqdm
from cloudvolume import CloudVolume
from similaritymeasures import frechet_dist
import seaborn as sns
from statannotations.Annotator import Annotator
from statannot import add_stat_annotation
from cloudvolume.exceptions import SkeletonDecodeError
import pickle
from copy import deepcopy
import networkx as nx
import os
from skimage import io
from matplotlib import cm
from matplotlib.colors import ListedColormap
from turtle import color
from scipy.ndimage import distance_transform_edt, binary_dilation
from joblib import Parallel, delayed
import networkx as nx
from brainlit.map_neurons.diffeo_gen import diffeo_gen_ara
from brainlit.map_neurons.map_neurons import Diffeomorphism_Transform

## Jacobian

### Initialize objects

In [ ]:
root_dir = Path(os.path.abspath(""))
data_dir = os.path.join(root_dir, "data")

velocity_path = os.path.join(data_dir, "downloop_1_v.mat")
affine_path = os.path.join(data_dir, "downloop_1_A.mat")

ct = CloudReg_Transform(velocity_path, affine_path, direction="atlas")

ara = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/ailey/ara/ara_10um.tif")

og_coords = ct.og_coords

xmin = np.amin(og_coords[0])
xmax = np.amax(og_coords[0])
ymin = np.amin(og_coords[1])
ymax = np.amax(og_coords[1])
zmin = np.amin(og_coords[2])
zmax = np.amax(og_coords[2])

print(f"x {xmax-xmin} y{ymax-ymin}-{ymax} z{zmax-zmin}")

Range of og_coords has padding of 190microns in total

### Functions for plotting

In [ ]:
def get_transformed_line(a, b, ct):
    ts = np.arange(0, 1.01, 0.01)
    line = np.zeros((len(ts), len(a)))
    for i, t in enumerate(ts):
        line[i, :] = a + t * (b - a)
    new_line = ct.evaluate(line)
    return new_line


def get_displacements(xs, ys, zs, ct):
    og_coords = np.meshgrid(xs, ys, zs, indexing="ij")
    og_coords = np.array(
        [og_coords[0].flatten(), og_coords[1].flatten(), og_coords[2].flatten()]
    ).T
    print(f"shp: {og_coords.shape}")
    new_coords = ct.evaluate(og_coords)
    diff = new_coords - og_coords

    return og_coords, diff


def get_border(im, border_id=0):
    if (im < 0).any():
        raise ValueError(f"Input image cannot have negative values")
    if border_id != 0:
        im[im == 0] = -1
        im[im == border_id] = 0
    edt = distance_transform_edt(im)
    edt[edt != 1] = 0
    return edt

### Plot displacement field over atlas

In [ ]:
slice_num = 600


fig, ax = plt.subplots()

xs = np.array([slice_num * 10 + 95])
ys = np.arange(ymin, ymax, 1000)
zs = np.arange(zmin, zmax, 1000)

og_coords, diff = get_displacements(xs, ys, zs, ct)

for y in ys:
    a = np.array([xs[0], y, zmin])
    b = np.array([xs[0], y, zmax])
    new_line = get_transformed_line(a, b, ct)
    # ax.plot(new_line[:,1], new_line[:,2], c='white')

for z in zs:
    a = np.array([xs[0], ymin, z])
    b = np.array([xs[0], ymax, z])
    new_line = get_transformed_line(a, b, ct)
    # ax.plot(new_line[:,1], new_line[:,2], c='white')

ax.scatter(og_coords[:, 1], og_coords[:, 2], label="Sample Space Grid", c="black")


ax.quiver(
    og_coords[:, 1],
    og_coords[:, 2],
    diff[:, 1],
    diff[:, 2],
    label="Scaled Displacement Field",
    color="black",
)
ax.axis("off")

# ARA slice

ara_slice = np.squeeze(ara[:, :, slice_num])
unqs = np.unique(ara_slice)


for i, unq in enumerate(unqs):
    if unq != 0:
        ara_slice[ara_slice == unq] = i
print(ara_slice.shape)

cmap = cm.get_cmap("tab20")
newcolors = cmap(np.arange(20))
newcolors = np.repeat(newcolors, 10, 0)
white = np.array([1, 1, 1, 1])
newcolors[0, :] = white
newcmp = ListedColormap(newcolors)
ax.imshow(ara_slice.T, extent=(-8000, 8000, -5000, 5000), cmap=newcmp)

### Plot Jacobian over atlas border

In [ ]:
z = 600
plt.rcParams.update({"font.size": 28})


fig, ax = plt.subplots(figsize=(15, 10))


# Determinant jacobians
xs = np.array([z])
ys = np.arange(ymin, ymax + 1, (ymax - ymin) / 100)
zs = np.arange(zmin, zmax + 1, (zmax - zmin) / 100)


Xs, Ys, Zs = np.meshgrid(xs, ys, zs)
XYZs = np.reshape(np.stack((Ys, Xs, Zs), axis=-1), newshape=(-1, 3))
detJs = []

for coord in tqdm(XYZs):
    J = ct.Jacobian(coord)
    detJs.append(np.log(np.linalg.det(J)))

detJs = np.array(detJs)
detJs = np.reshape(detJs, newshape=Xs.shape)
print(detJs.shape)


newcolors = np.array([[1, 1, 1, 0], [0, 0, 0, 1]])
cmap_trans_white = ListedColormap(newcolors)

max_abs = np.amax(np.abs(detJs))
plt_ldjs = ax.imshow(
    np.squeeze(detJs).T,
    extent=(-8000, 8000, -5000, 5000),
    cmap="seismic",
    vmin=-max_abs*2,
    vmax=max_abs*2,
)
ax.axis("off")
#fig.colorbar(plt_ldjs, ax=ax)

# Displacement
ys = np.arange(ymin, ymax + 1, (ymax - ymin) / 8)
zs = np.arange(zmin, zmax + 1, (zmax - zmin) / 8)
og_coords, diff = get_displacements(xs, ys, zs, ct)

ax.scatter(og_coords[:, 1], og_coords[:, 2], c="black")


ax.quiver(
    og_coords[:, 1],
    og_coords[:, 2],
    diff[:, 1],
    diff[:, 2],
    label="Scaled Displacement Field",
    color="black",
)


# ARA border

ara_slice = np.squeeze(ara[:, :, z])
border = get_border(ara_slice)
border = border.T
border = binary_dilation(border, iterations=2)
plt_border = ax.imshow(border, extent=(-8000, 8000, -5000, 5000), cmap=cmap_trans_white)

#ax.legend(loc="lower right", bbox_to_anchor=(1, -0.1))
#fig.suptitle("Log Jacobian Determinant of Atlas to Target Mapping")

## Find max jacobian

In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
# Determinant jacobians
grid = "cust-full"
xmin, xmax = -1890, -250
ymin, ymax = -2400, -1600
zmin, zmax = 800, 2000
if grid == "cust-full":
    xs = np.arange(xmin, xmax + 1, (xmax - xmin) / 50)
    ys = np.arange(ymin, ymax + 1, (ymax - ymin) / 50)
    zs = np.arange(zmin, zmax + 1, (zmax - zmin) / 50)
    Xs, Ys, Zs = np.meshgrid(xs, ys, zs)
elif grid == "full":
    Xs, Ys, Zs = og_coords[0], og_coords[1], og_coords[2]
elif grid == "cust-box":
    xmin, xmax = -1950, -250
    ymin, ymax = -2450, -1650
    zmin, zmax = 750, 1950
    xs = np.arange(xmin, xmax + 1, (xmax - xmin) / 50)
    ys = np.arange(ymin, ymax + 1, (ymax - ymin) / 50)
    zs = np.arange(zmin, zmax + 1, (zmax - zmin) / 50)
    Xs, Ys, Zs = np.meshgrid(xs, ys, zs)

XYZs = np.reshape(np.stack((Ys, Xs, Zs), axis=-1), newshape=(-1, 3))
norms = []


def norm_diff(coord):
    diff = ct.Jacobian(coord) - np.eye(3)

    return np.linalg.norm(diff)


norms = Parallel(n_jobs=10)(delayed(norm_diff)(coord) for coord in tqdm(XYZs))


max_abs = np.amax(norms)

rad = np.linalg.norm([xmax - xmin - 190, ymax - ymin - 190, zmax - zmin - 190])
print(f"Max |J-I|: {max_abs}, rad {rad} = {rad*max_abs}")

In [ ]:
def displacement(coord):
    diff = ct.evaluate(coord) - coord

    return np.linalg.norm(diff)


displacements = Parallel(n_jobs=10)(delayed(norm_diff)(coord) for coord in tqdm(XYZs))

max_displacement = np.amax(displacements)
print(f"Max |ei-ei-1|: {2*max_displacement}")

In [ ]:
xs = np.arange(0, 3000)
ys = max_abs * xs / 2 + max_displacement

print(f"Bound: Zeroth Order Mapping Error <= {max_abs/2}L + {max_displacement} ")

plt.plot(xs, ys)

## Spectral norms of jacobians

In [ ]:
# spectral norm jacobians
xs = np.arange(xmin, xmax + 1, (xmax - xmin) / 25)
ys = np.arange(ymin, ymax + 1, (ymax - ymin) / 25)
zs = np.arange(zmin, zmax + 1, (zmax - zmin) / 25)


Xs, Ys, Zs = np.meshgrid(xs, ys, zs)
XYZs = np.reshape(np.stack((Ys, Xs, Zs), axis=-1), newshape=(-1, 3))
normJs = []

for coord in tqdm(XYZs):
    J = ct.Jacobian(coord)
    normJs.append(np.linalg.norm(J))

In [ ]:
plt.hist(normJs)

In [ ]:
def Fx(pos):
    pos = np.array(pos)
    if len(pos.shape) == 1:
        return pos[0]
    elif len(pos.shape) == 2:
        return pos[:, 0]


def Fy(pos):
    pos = np.array(pos)
    if len(pos.shape) == 1:
        return pos[1]
    elif len(pos.shape) == 2:
        return pos[:, 1]


def Fz(pos):
    pos = np.array(pos)
    if len(pos.shape) == 1:
        return pos[2]
    elif len(pos.shape) == 2:
        return pos[:, 2]


og_diffeo = ct.diffeomorphism
ct.diffeomorphism = (Fx, Fy, Fz)


fig, ax = plt.subplots()


# Determinant jacobians
xs = np.array([0])
ys = np.arange(-5, 6)
zs = np.arange(-5, 6)


Xs, Ys, Zs = np.meshgrid(xs, ys, zs)
XYZs = np.reshape(np.stack((Ys, Xs, Zs), axis=-1), newshape=(-1, 3))
detJs = []

for coord in tqdm(XYZs):
    J = ct.Jacobian(coord)
    detJs.append(np.log(np.linalg.det(J)))

detJs = np.array(detJs)
detJs = np.reshape(detJs, newshape=Xs.shape)
print(detJs.shape)


newcolors = np.array([[1, 1, 1, 0], [0, 0, 0, 1]])
cmap_trans_white = ListedColormap(newcolors)

max_abs = np.amax(np.abs(detJs))
plt_ldjs = ax.imshow(
    np.squeeze(detJs).T,
    extent=(-5, 5, -5, 5),
    cmap="seismic",
    vmin=-max_abs,
    vmax=max_abs,
)
ax.axis("off")
fig.colorbar(plt_ldjs, ax=ax)

# Displacement
og_coords, diff = get_displacements(xs, ys, zs, ct)

ax.scatter(og_coords[:, 1], og_coords[:, 2], label="Sample Space Grid", c="black")


ax.quiver(
    og_coords[:, 1],
    og_coords[:, 2],
    diff[:, 1],
    diff[:, 2],
    label="Scaled Displacement Field",
    color="black",
)


ct.diffeomorphism = og_diffeo

## Neuron Mapping

In [ ]:
root_dir = Path(os.path.abspath(""))
data_dir = os.path.join(root_dir, "data", "mapping-files")

In [ ]:
velocity_path = os.path.join(data_dir, "downloop_1_v.mat")
affine_path = os.path.join(data_dir, "downloop_1_A.mat")

ct = CloudReg_Transform(velocity_path, affine_path)

im_path = "precomputed://file://" + os.path.join(data_dir, "ch1_otsu_iso")
vol_im = CloudVolume(im_path)
shp = np.array(vol_im.shape)
res_im = np.array(vol_im.resolution) / 1000
origin_im = (shp[:3] - 1) * res_im / 2

axons_path = "precomputed://file://" + os.path.join(data_dir, "axons")
vol = CloudVolume(axons_path)
shp = np.array(vol.shape)
res_atlas = np.array(vol.resolution) / 1000
origin_atlas = (shp[:3] - 1) * res_atlas / 2

In [ ]:
# 1,22; 4,179
neuron_id = 1
branch_id = -1

# get coords in proper cooordinates
skel = vol.skeleton.get(neuron_id)
coords = skel.vertices / 1000 - origin_im

# apply affine transform
coords = ct.apply_affine(coords)

G = GeometricGraph()
for id, coord in enumerate(coords):
    G.add_node(id, loc=coord)
for edge in skel.edges:
    G.add_edge(edge[0], edge[1])

spline_tree = G.fit_spline_tree_invariant()

# Target space

fig = plt.figure(figsize=(18, 10), dpi=300)
ax = fig.add_subplot(1, 2, 1, projection="3d")

G_transformed = deepcopy(G)
G_transformed = transform_geometricgraph(G_transformed, ct, deriv_method="two-sided")
soma = np.array(G.nodes[G.root]["loc"])
spline_tree = G.spline_tree

for i, node in enumerate(tqdm(spline_tree.nodes, desc="Target space")):
    if node != branch_id and branch_id != -1:
        continue
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    tck = spline[0]

    # trace points only
    pts = splev(u, tck)
    if i == 0 or node == branch_id:
        label = "Neuron Branch"
    else:
        label = None
    ax.plot(pts[0], pts[1], pts[2], linestyle="-", label=label, color="red")
    derivs = splev(u, tck, der=1)

# Plot displacement field
xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()
zmin, zmax = ax.get_zlim()
xs = np.arange(xmin, xmax, (xmax - xmin) / 4)
ys = np.arange(ymin, ymax, (ymax - ymin) / 4)
zs = np.arange(zmin, zmax, (zmax - zmin) / 4)

og_coords = np.meshgrid(xs, ys, zs, indexing="ij")
og_coords = np.array(
    [og_coords[0].flatten(), og_coords[1].flatten(), og_coords[2].flatten()]
).T
new_coords = ct.evaluate(og_coords)

displacements = new_coords - og_coords
ax.quiver(
    og_coords[:, 0],
    og_coords[:, 1],
    og_coords[:, 2],
    displacements[:, 0],
    displacements[:, 1],
    displacements[:, 2],
    length=20,
    label="Scaled Displacement Field",
    alpha=0.75,
)  # length

ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

#ax.legend(fontsize=12)
plt.axis('off')

ax = fig.add_subplot(1, 2, 2, projection="3d")

for i, node in enumerate(tqdm(spline_tree.nodes, desc="0th order mappings")):
    if node != branch_id and branch_id != -1:
        continue
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    tck = spline[0]
    pts = splev(u, tck)
    pts = np.stack(pts, axis=1)

    # dense line points
    tck_line, u_line = splprep(pts.T, k=1, s=0)
    u_line = np.arange(u_line[0], u_line[-1] + 0.01, 0.01)
    pts_line = splev(u_line, tck_line)
    pts_line = np.stack(pts_line, axis=1)
    trans_pts = ct.evaluate(pts_line)
    if i == 0 or node == branch_id:
        label = "Continuous Mapping (Ground Truth)"
    else:
        label = None
    # ax.plot(
    #     trans_pts[:, 0],
    #     trans_pts[:, 1],
    #     trans_pts[:, 2],
    #     linestyle="-",
    #     color="red",
    #     label=label,
    # )

    # Transformed points
    trans_pts = ct.evaluate(pts)
    if i == 0 or node == branch_id:
        label = "Discrete Mapping - 0th Order"
    else:
        label = None
    # ax.plot(
    #     trans_pts[:, 0],
    #     trans_pts[:, 1],
    #     trans_pts[:, 2],
    #     linestyle="-",
    #     label=label,
    #     color="blue",
    #     alpha=0.5,
    # )
    derivs = splev(u, tck, der=1)
    derivs = np.stack(derivs, axis=1)
    trans_derivs = ct.D(pts, derivs)

# act on derivatives
soma = np.array(G_transformed.nodes[G_transformed.root]["loc"])
spline_tree = G_transformed.spline_tree
for i, node in enumerate(tqdm(spline_tree.nodes, desc="1st order mappings")):
    if node != branch_id and branch_id != -1:
        continue
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    u = np.arange(u[0], u[-1] + 0.01, 0.01)
    chspline = spline[0]
    pts = chspline(u)

    if i == 0 or node == branch_id:
        label = "Discrete Mapping - 1st Order"
    else:
        label = None
    ax.plot(pts[:, 0], pts[:, 1], pts[:, 2], linestyle="-", label=label, color="green")

ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(True)
#ax.legend(fontsize=12)
plt.axis('off')
plt.show()

## Soma Mapping

Neuroglancer coordinates: in voxels (with voxel size printed)

Cloudvolume skeleton coordinates: in nanometers

SWC: in microns - add offset, then subtract origin of image (in transform.txt), which is in nanometers

In [ ]:
root_dir = Path(os.path.abspath(""))
data_dir = os.path.join(root_dir, "data", "mapping-files")
swc_dir = os.path.join(data_dir, "all-swcs")

In [ ]:
velocity_path = os.path.join(data_dir, "downloop_1_v.mat")
affine_path = os.path.join(data_dir, "downloop_1_A.mat")

ct = CloudReg_Transform(velocity_path, affine_path)

im_path = "precomputed://file://" + os.path.join(data_dir, "ch1_otsu_iso")
vol_im = CloudVolume(im_path)
shp = np.array(vol_im.shape)
res_im = np.array(vol_im.resolution) / 1000
origin_im = (shp[:3] - 1) * res_im / 2

axons_path = "precomputed://file://" + os.path.join(data_dir, "axons")
vol = CloudVolume("precomputed://https://open-neurodata.s3.amazonaws.com/ara_2016/sagittal_10um/annotation_10um_2017")
shp = np.array(vol.shape)
res_atlas = np.array(vol.resolution) / 1000
origin_atlas = (shp[:3] - 1) * res_atlas / 2

image_origin = np.zeros(3)
coord_to_pos = {"x":0,"y":1,"z":2}
with open(os.path.join(swc_dir, "transform.txt")) as f:
    for line in f.readlines():
        if "o" == line[0]:
            image_origin[coord_to_pos[line[1]]] = float(line.split(":")[1])

image_origin /= 1000

In [ ]:
file_list = os.listdir(swc_dir)

soma_coords = []
for file in file_list:
    if ".swc" in file:
        trace = NeuronTrace(os.path.join(swc_dir, file), rounding=False, read_offset=True)
        point = trace.df.loc[0]
        if point['parent'] >= 0:
            print(f"Could not find root node in {file}")
        else:
            soma_coord = np.array([point['x'], point['y'], point['z']])
            soma_coord = soma_coord - image_origin
            soma_coords.append(soma_coord)
            #print(f"{file}: {soma_coord/[2.92,2.92,5]}")
soma_coords_target_microns = np.array(soma_coords)

## What is registration

In [ ]:
vol = CloudVolume(
    "precomputed://https://open-neurodata.s3.amazonaws.com/ara_2016/sagittal_25um/average_25um"
)
im_og = np.array(vol[:, :, :])

In [ ]:
d = np.arange(im_og.shape[0])
v = np.arange(im_og.shape[1])
h = np.arange(im_og.shape[2])

image_interp = RegularGridInterpolator(
    (d, v, h), im_og, bounds_error=False, fill_value=0
)

im_og_slice = im_og[280, :, :]
plt.imshow(im_og_slice, cmap='gray')
print(im_og.shape)

In [ ]:
im_og_slice_blur = gaussian_filter(im_og[29, :, :], sigma=15)
#plt.imshow(im_og[70, :, :], cmap='gray')
plt.imshow(im_og_slice_blur, cmap='gray')


In [ ]:
# x is vertical component, y is horizontal
def deformation(h_atlas, v_atlas):
    y_disp = np.exp(-1 * v_atlas / 200) * np.exp(-1 * (h_atlas - 228) ** 2 / 10000) * 100
    return y_disp

k = 70
c2 = 0.01
m = 228
c1 = 0.0005
def deformation(h_atlas, v_atlas):
    v_target = v_atlas + k/(c2*v_atlas+c1*(h_atlas-m)**2+1)
    return v_target - v_atlas


def deform_inv(h_target,v_target):
    h_atlas = h_target
    c_combine = 1 + c1*(h_atlas-m)**2
    b = c_combine-v_target*c2
    c = k - v_target*c_combine
    a = c2
    disc = b**2-4*a*c
    v_atlas = (-b+np.sqrt(disc))/(2*a)

    return v_target - v_atlas


k=50
f = 1/50
def deformation(h_atlas, v_atlas):
    v_target = v_atlas + k*np.sin(f*h_atlas)*np.exp(-(h_atlas-300)**2/20000)
    return v_target - v_atlas


def deform_inv(h_target,v_target):
    h_atlas = h_target
    v_atlas = v_target - k*np.sin(f*h_atlas)*np.exp(-(h_atlas-300)**2/20000)

    return v_target - v_atlas

print(deformation(228, 0))
deform_inv(228, 0+deformation(228, 0))

In [ ]:
v_mat, h_mat = np.meshgrid(np.arange(0, im_og.shape[1], im_og.shape[1]/11), np.arange(0, im_og.shape[2], im_og.shape[2]/11))
v_flat = np.reshape(v_mat, (-1, 1))
h_flat = np.reshape(h_mat, (-1, 1))
y_displacements = deformation(h_flat, v_flat)
y_displacements = np.reshape(y_displacements, v_mat.shape)

plt.imshow(im_og[280, :, :], cmap="gray")
plt.quiver(
    h_mat,
    v_mat,
    0*y_displacements,
    -y_displacements,
    scale=1,
    scale_units="x",
    facecolor="red",
)

In [ ]:
v_mat, h_mat, d_mat = np.meshgrid(v, h, d)

print(v_mat.shape)
v_flat = np.reshape(v_mat, (-1))
h_flat = np.reshape(h_mat, (-1))
d_flat = np.reshape(d_mat, (-1))

v_displacements = deformation(h_flat, v_flat)
new_v = v_flat - v_displacements

In [ ]:
points = np.stack([d_flat, new_v, h_flat], axis=1)
new_im_flat = image_interp(points)
new_im = np.reshape(new_im_flat, v_mat.shape)
new_im = np.swapaxes(new_im, 0, -1)
new_im = new_im.astype("uint16")
plt.imshow(new_im[280, :, :], cmap="gray")

In [ ]:
v_mat, h_mat = np.meshgrid(np.arange(0, im_og.shape[1]+1, im_og.shape[1]/11), np.arange(0, im_og.shape[2]+1, im_og.shape[2]/11))
v_flat = np.reshape(v_mat, (-1, 1))
h_flat = np.reshape(h_mat, (-1, 1))
y_displacements = deform_inv(h_flat, v_flat)
y_displacements = np.reshape(y_displacements, v_mat.shape)

print(v_flat.shape)

fig = plt.figure(figsize=(10,10))
plt.imshow(im_og[280, :, :], cmap="gray")
for row in np.arange(y_displacements.shape[0]):
    xs = h_mat[:,row]
    ys = v_mat[:,row] - y_displacements[:,row]
    plt.plot(xs,ys, c='red')
for col in np.arange(y_displacements.shape[0]):
    xs = h_mat[col,:]
    ys = v_mat[col,:]
    plt.plot(xs,ys, c='red')

## Techniques

In [202]:

tools = ['Generative Diffeomorphic Mapping','mBrainAligner','Multiscale Varifold LDDMM', 'Novel affine methods', 'ANTs', 'CloudReg', 'Elastic Alignment', 'Elastix', 'Custom MATLAB scripts', 'quickNII/Visualign', 'TissueCyte', 'Warp3', 'nTracer2']
models = ['Affine','Polynomial','Spline', 'Elastic', 'Manual Pinning of Points', 'LDDMM']
metrics = ['Squared Error (SE)','SE with contrast change','Landmark SE', 'Varifold norm', 'Correlation', 'Mutual information', 'Manual placement']

tools_models_edges = [('Generative Diffeomorphic Mapping','Affine'), ('Generative Diffeomorphic Mapping','LDDMM'),
                      ('Elastix','Affine'), ('Elastix','Spline'),
                      ('CloudReg','Affine'), ('CloudReg','LDDMM'), ('mBrainAligner','Spline'),
                      ('ANTs','Affine'), ('ANTs','Spline'), ('ANTs','LDDMM'),
                      ('TissueCyte', 'Affine'), ('quickNII/Visualign', 'Affine'),
                      ('Custom MATLAB scripts', 'Affine'), ('Custom MATLAB scripts', 'Polynomial'), ('Custom MATLAB scripts','Spline'),
                      ('Multiscale Varifold LDDMM', 'LDDMM'),('Novel affine methods', 'Affine'),
                      ('Elastic Alignment', 'Elastic')]
tools_metrics_edges = [('Generative Diffeomorphic Mapping','SE with contrast change'), ('Elastix','Mutual information'), ('Elastix','Correlation'), ('Elastix','Squared Error (SE)'),
                       ('CloudReg','SE with contrast change'),
                       ('mBrainAligner','Squared Error (SE)'), ('mBrainAligner','Correlation'), ('mBrainAligner','Mutual information'),
                       ('quickNII/Visualign', 'Correlation'),('quickNII/Visualign', 'Manual placement'),('Multiscale Varifold LDDMM', 'Varifold norm'),
                       ('Novel affine methods', 'Squared Error (SE)'),
                       ('ANTs', 'Squared Error (SE)'), ('ANTs', 'Correlation'), ('ANTs', 'Mutual information'),
                       ('Custom MATLAB scripts', 'Squared Error (SE)'), ('Custom MATLAB scripts', 'Mutual information')]





In [ ]:
BG = nx.Graph()
BG.add_nodes_from(models)
BG.add_nodes_from(tools)
BG.add_nodes_from(metrics)

BG.add_edges_from(tools_models_edges)
BG.add_edges_from(tools_metrics_edges)

pos = dict()
factor = 12
middle_vert_pos = [factor*(len(tools) - i) for i in range(len(tools))]
middle_vert_pos = [p+2*factor if i < 4 else p for i,p  in enumerate(middle_vert_pos)]
middle_max = np.amax(middle_vert_pos)

models_shift = (middle_max-factor*(len(models)-1))/2
pos.update((n, (1,factor*(len(models)-i)+models_shift)) for i, n in enumerate(models))
pos.update((n, (2,p)) for n, p in zip(tools,middle_vert_pos))
metrics_shift = (middle_max-factor*(len(metrics)-1))/2
pos.update((n, (3,factor*(len(metrics)-i)+metrics_shift)) for i, n in enumerate(metrics))

fig, ax = plt.subplots(figsize=(12,10))
labels = nx.draw_networkx_labels(BG, pos=pos,  verticalalignment='center', bbox=dict(facecolor='white'), clip_on=False)
nx.draw(BG, pos=pos, ax=ax, node_size=200, node_shape='s')

### Citations
ntracer2 - dawen - asked, ask about elastic tool or novel affine method?
novel affine, elastic - dawen??

generative diffeomorphic mapping - daniel citation - Tward D, Brown T, Kageyama Y, Patel J, Hou Z, Mori S, Albert M, Troncoso J, Miller M. Diffeomorphic registration with intensity transformation and missing data: Application to 3D digital pathology of Alzheimer's disease. Frontiers in neuroscience. 2020 Feb 11;14:52.

Tward D, Li X, Huo B, Lee B, Miller MI, Mitra PP. Solving the where problem in neuroanatomy: a generative framework with learned mappings to register multimodal, incomplete data into a reference brain. bioRxiv. 2020 Mar 23:2020-03.

multiscale varifold lddmm - https://arxiv.org/pdf/2208.08376.pdf
cloudreg - https://www.nature.com/articles/s41592-021-01218-z - done
mbrainaligner - https://academic.oup.com/bioinformatics/article/38/19/4654/6661343 Hanchuan - done
ANTS - http://stnava.github.io/ANTs/ - done
elastix - https://pubmed.ncbi.nlm.nih.gov/19923044/ - done
matlab - https://www.mathworks.com/matlabcentral/answers/414438-how-do-i-cite-matlab-in-a-bibliography-or-a-published-journal#:~:text=Citing%20MATLAB%20software%3A&text=Citation%20in%20Vancouver%20style%3A,The%20MathWorks%20Inc.%3B%202022. https://www.mathworks.com/help/images/ref/imregister.html#d124e180774
quickNII/Visualign - https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0216796 https://www.nitrc.org/projects/visualign
tissuecyte - cite software https://www.tissuevision.com/approach
Warp3 - https://www.nature.com/articles/s41467-021-22915-5 dong - asked


In [203]:
# tools_metrics_edges += ("Matlab", "Landmark SSE")

metrics += ["Unknown metric"]
models += ["Unknown model"]
tools_metrics_edges += [(tool, metrics[-1]) for tool in ["Elastic Alignment", "TissueCyte", "Warp3"]]
tools_models_edges += [(tool, models[-1]) for tool in ["Warp3"]]

In [204]:
import plotly.graph_objects as go

labels = tools + models + metrics


xs = [0.35 for t in tools] + [0.02 for m in models] + [0.9 for m in metrics]
ys = list(np.arange(0.1, 0.09*(len(tools)+1), 0.09)) + list(np.arange(0.1, 0.09*(len(models)+1), 0.09)) + list(np.arange(0.1, 0.09*(1+len(metrics)), 0.09))

color = ["mistyrose", "moccasin", "khaki", "palegreen", "aquamarine", "powderblue", "cyan", "thistle", "orchid", "goldenrod", "silver", "lightgrey"]
tool2color = {k:v for (k,v) in zip(tools, color)}
color += ["black" for t in models]
color += ["black" for m in metrics]

source = []
target = []
e_color = []
for tool_model_edge in tools_models_edges:
  source.append(labels.index(tool_model_edge[1]))
  target.append(labels.index(tool_model_edge[0]))
  e_color.append(tool2color[tool_model_edge[0]])

for tool_metric_edge in tools_metrics_edges:
  source.append(labels.index(tool_metric_edge[0]))
  target.append(labels.index(tool_metric_edge[1]))
  e_color.append(tool2color[tool_metric_edge[0]])

value = [1 for s in source]

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = color,
      #x=xs,
      #y=ys
    ),
    link = dict(
      source = source, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = target,
      value = value,
      color = e_color
  ))])

fig.update_layout(font_size=24)
fig.update_layout(width=1150, height=1000)
fig.show()

In [175]:
from ngauge import Neuron

In [185]:
n = Neuron.from_swc("/Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/mouselight-swcs/swcs-1/AA1087.swc")

In [190]:
for b in n.branches:
    print(np.amax(b.path_dist_to_ends())/10000)
n.total_bif_nodes()


1.4080889430451775
0.034744588499389974
0.016900518965864735
0.03365819186590806
0.021338873390637995


34